## DIA 043: Implementación de Alertas Automatizadas con Slack

En el Día 43 se añade una capa adicional de monitoreo a la API mediante la integración con Slack. Esta solución utiliza un webhook de Slack para enviar notificaciones automáticas a un canal predefinido cada vez que se produce un evento crítico (por ejemplo, errores en el endpoint o situaciones que requieran intervención inmediata). La integración se realiza a través de una función que envía un mensaje a Slack y un endpoint protegido para probar y validar la funcionalidad. Esto ayuda a mantener a los administradores informados en tiempo real, mejorando la capacidad de respuesta ante incidentes.

Código Completo (api.py)
python
Copiar
import os
import io
import random
import json
import time
import threading
import logging
from datetime import datetime
from functools import wraps
import requests

from flask import Flask, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room
from flasgger import Swagger

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Configuración de Flasgger para documentación (opcional)
swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": "apispec_1",
            "route": "/apispec_1.json",
            "rule_filter": lambda rule: True,
            "model_filter": lambda tag: True,
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}
swagger = Swagger(app, config=swagger_config)

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos (simplificados)
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos para este ejemplo

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Decorador para Roles (simplificado)
# ---------------------------
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            if getattr(user, 'role', 'user') != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Función para Enviar Notificaciones a Slack
# ---------------------------
def send_slack_notification(message):
    """
    Envía un mensaje a Slack usando un webhook configurado en la variable de entorno SLACK_WEBHOOK_URL.
    """
    webhook_url = os.getenv("SLACK_WEBHOOK_URL")
    if not webhook_url:
        logger.error("SLACK_WEBHOOK_URL no está configurado.")
        return False
    payload = {"text": message}
    headers = {"Content-Type": "application/json"}
    response = requests.post(webhook_url, data=json.dumps(payload), headers=headers)
    if response.status_code != 200:
        logger.error("Error al enviar notificación a Slack: %s", response.text)
        return False
    return True

# ---------------------------
# Endpoints Comunes
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    """
    Endpoint para el login de usuarios.
    ---
    tags:
      - Auth
    parameters:
      - in: body
        name: credentials
        schema:
          type: object
          required:
            - username
            - password
          properties:
            username:
              type: string
            password:
              type: string
    responses:
      200:
        description: Token de acceso JWT generado.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

@app.route('/health', methods=['GET'])
def health():
    """
    Health Check
    ---
    tags:
      - Health
    responses:
      200:
        description: La aplicación está en funcionamiento.
    """
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoints de Versionado (v1 y v2) – Se mantienen para referencia
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    logger.info("v1: Predicción realizada.")
    return jsonify(result), 200

api_v2 = Blueprint('api_v2', __name__)
@api_v2.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("150 per day")
def predict_v2():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    result = {"prediccion": 7, "probabilidad": 0.95, "version": "v2", "mensaje": "Predicción mejorada"}
    logger.info("v2: Predicción realizada con mejoras.")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')
app.register_blueprint(api_v2, url_prefix='/api/v2')

# ---------------------------
# Endpoint de A/B Testing para la Predicción (para referencia)
# ---------------------------
ab_blueprint = Blueprint('api_ab', __name__)
@ab_blueprint.route('/predict', methods=['POST'])
@jwt_required()
def predict_ab():
    version = random.choice(['v1', 'v2'])
    if version == 'v1':
        response = predict_v1()
    else:
        response = predict_v2()
    data = response.get_json()
    data['ab_version'] = version
    logger.info(f"A/B Testing: Solicitud enrutada a la versión {version}.")
    return jsonify(data), response.status_code

app.register_blueprint(ab_blueprint, url_prefix='/api/ab')

# ---------------------------
# Endpoint de Retraining con MLFlow (para referencia)
# ---------------------------
@app.route('/admin/retrain_mlflow', methods=['POST'])
@jwt_required()
@role_required('admin')
def retrain_mlflow():
    mlflow.set_experiment("Retraining Experiment")
    with mlflow.start_run() as run:
        logger.info("Inicio del retraining con MLFlow...")
        time.sleep(10)  # Simula el tiempo de retraining
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_metric("accuracy", 0.92)
        artifact_path = "model_info.txt"
        with open(artifact_path, "w") as f:
            f.write("Modelo actualizado basado en feedback con MLFlow.")
        mlflow.log_artifact(artifact_path)
        logger.info("Retraining completado y registrado en MLFlow.")
        return jsonify({"msg": "Retraining experiment logged in MLFlow", "run_id": run.info.run_id}), 202

# ---------------------------
# Nuevo Endpoint: Envío de Notificación Slack para Alertas
# ---------------------------
@app.route('/admin/slack_alert', methods=['POST'])
@jwt_required()
@role_required('admin')
def slack_alert():
    """
    Endpoint para enviar una notificación de alerta a Slack.
    ---
    tags:
      - Alerts
    parameters:
      - in: body
        name: alert
        schema:
          type: object
          required:
            - message
          properties:
            message:
              type: string
    responses:
      200:
        description: Notificación enviada exitosamente.
      500:
        description: Error al enviar la notificación.
    """
    data = request.get_json()
    message = data.get("message", "Alerta de la API")
    if send_slack_notification(message):
        return jsonify({"msg": "Slack alert sent successfully"}), 200
    else:
        return jsonify({"msg": "Failed to send Slack alert"}), 500

# ---------------------------
# Endpoint de Reporte Automatizado (para referencia)
# ---------------------------
@app.route('/admin/report', methods=['GET'])
@jwt_required()
@role_required('admin')
def generate_report():
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No hay feedback disponible"}), 200
    correct_feedback = Feedback.query.filter_by(correct=True).count()
    incorrect_feedback = Feedback.query.filter_by(correct=False).count()
    accuracy = (correct_feedback / total_feedback) * 100
    report = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_feedback,
        "incorrect_feedback": incorrect_feedback,
        "accuracy_percentage": accuracy
    }
    logger.info("Reporte generado: " + json.dumps(report))
    return jsonify(report), 200

# ---------------------------
# Endpoint de Health Check (para referencia)
# ---------------------------
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Ejecutar la aplicación con soporte para WebSockets y Swagger
# ---------------------------
if __name__ == '__main__':
    # En producción se recomienda usar socketio.run(app, debug=False)
    socketio.run(app, debug=True)
Explicación del Código
Integración de MLFlow para Retraining:
El endpoint /admin/retrain_mlflow inicia un proceso de retraining simulado y registra los parámetros, métricas y artefactos del experimento en MLFlow. Esto facilita el seguimiento de los experimentos de retraining para mejorar el modelo.

Endpoint de Notificaciones Slack:
La función send_slack_notification envía un mensaje a un webhook configurado en la variable de entorno SLACK_WEBHOOK_URL. El endpoint /admin/slack_alert permite a los administradores enviar una alerta a Slack para notificar incidentes o eventos críticos.

Endpoints de Versionado y A/B Testing:
Se mantienen endpoints para las versiones v1 y v2 de /predict y se agrega un endpoint de A/B testing que distribuye solicitudes entre ambas versiones de forma aleatoria.

Endpoints Comunes:
Se incluyen endpoints para login, health check, feedback, reportes y retraining, que ya han sido implementados en días anteriores, integrados en un solo archivo para referencia.

Ejecución de la Aplicación:
La aplicación se ejecuta con Flask-SocketIO, permitiendo soporte tanto para HTTP como para WebSockets. Además, Flasgger genera la documentación en la ruta /docs/ (aunque en este ejemplo la configuración de Flasgger se incluye en días anteriores).

